In [2]:
!pip3 install scikit-learn


131.38s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip3 install seaborn


139.56s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
!pip3 install numpy

147.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
!pip3 install torch torchvision torchaudio


158.86s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [6]:
# Import torch and neural network library 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset  


# import sklearn model_selection, StandardScaler
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import numpy, pandas, matplotlib, seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Setup device either gpu or cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


print('Device used: ', device)

Device used:  cpu


In [7]:
# read the dataset framingham.csv and display the first 5 rows.
data = pd.read_csv('/Users/suarezcruzrosy/Downloads/framingham.csv')
data.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [9]:
# display the shape, null values
data.shape, data.isnull().sum()


((4240, 16),
 male                 0
 age                  0
 education          105
 currentSmoker        0
 cigsPerDay          29
 BPMeds              53
 prevalentStroke      0
 prevalentHyp         0
 diabetes             0
 totChol             50
 sysBP                0
 diaBP                0
 BMI                 19
 heartRate            1
 glucose            388
 TenYearCHD           0
 dtype: int64)

In [10]:
# Fill null values with either median or mean.
data['education'] = data['education'].fillna(data['education'].median())
data['cigsPerDay'] = data['cigsPerDay'].fillna(data['cigsPerDay'].mean())
data['BPMeds'] = data['BPMeds'].fillna(data['BPMeds'].mean()) 
data['totChol'] = data['totChol'].fillna(data['totChol'].mean())
data['BMI'] = data['BMI'].fillna(data['BMI'].mean()) 
data['glucose'] = data['glucose'].fillna(data['glucose'].mean())
data['heartRate'] = data['heartRate'].fillna(data['heartRate'].mean())  
data.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [59]:
# get the features as X and the target as y
X_features = data.drop(['diabetes'], axis=1)
y_target = data['diabetes']

In [60]:
# Split the data into train and test:
x_train , x_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.2, random_state=42)

In [61]:
# Scale the features X_train and X_test using StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(x_train)
X_test = scaler.transform(x_test)


# Convert the X_train, X_test, y_train, y_test to torch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.float32)



In [62]:
# Build your neural network
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [63]:
# Instantiate the model, define the loss function and the optimizer
model = NeuralNetwork()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)    


In [64]:
# Train the model

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")  


In [65]:
!pip3 install --upgrade torch torchvision


2558.99s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [66]:
!pip uninstall -y numpy
!pip install numpy



2568.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Found existing installation: numpy 2.2.4
Uninstalling numpy-2.2.4:
  Successfully uninstalled numpy-2.2.4


2577.78s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached numpy-2.2.4-cp313-cp313-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp313-cp313-macosx_14_0_x86_64.whl (6.7 MB)


In [74]:
# Evaluate the model
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X = torch.randn(100, 10)
y = torch.randint(0, 2, (100,))
dataset = TensorDataset(X, y)
test_dataloader = DataLoader(dataset, batch_size=10)

model = nn.Sequential(
    nn.Linear(10, 16),
    nn.ReLU(),
    nn.Linear(16, 2)
).to(device)

loss_fn = nn.CrossEntropyLoss()

def test(dataloader, model, loss_fn):        
    print("Entered the test function")
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error:\n Accuracy: {(100 * correct):.1f}%, Avg loss: {test_loss:.6f}\n")

print("Calling test function now...")
test(test_dataloader, model, loss_fn)
print("Test function finished.")

    



Calling test function now...
Entered the test function
Test Error:
 Accuracy: 50.0%, Avg loss: 0.701684

Test function finished.
